In [1]:
library(tidyverse)
library(tidymodels)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.0 ──

✔ broom        1.0.4     ✔ rsample      1.1.1
✔ dials        1.2.0     ✔ tune         1.1.1
✔ infer        1.0.4     ✔ workflows    1.1.3
✔ modeldata    1.1.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.0     ✔ yardstick    1.2.0
✔ recipes      1.0.6     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [11]:
chr_training <- read_csv("data/chr_training.csv") |>
    mutate(label = factor(label))
chr_testing <- read_csv("data//chr_testing.csv") |>
    mutate(label = factor(label))

head(chr_training)
head(chr_testing)

Rows: 1950 Columns: 785
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (1): label
dbl (784): 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 650 Columns: 785
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (1): label
dbl (784): 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


label,1,2,3,4,5,6,7,8,9,⋯,775,776,777,778,779,780,781,782,783,784
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


label,1,2,3,4,5,6,7,8,9,⋯,775,776,777,778,779,780,781,782,783,784
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [5]:
chr_knn_recipe <- recipe(label ~ ., data = chr_training)


chr_knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

gridvals <- tibble(neighbors = seq(from = 1, to = 50, by = 1))

chr_vfold <- vfold_cv(chr_training, v = 5, strata = label)

chr_wf <- workflow() |>
    add_recipe(chr_knn_recipe) |>
    add_model(chr_knn_spec) |>
    tune_grid(resamples = chr_vfold, grid = gridvals)

chr_result <- chr_wf |>
    collect_metrics() |>
    filter(.metric == "accuracy")

Warning message:
“Too little data to stratify.
• Resampling will be unstratified.”


In [10]:
best_k <- chr_result |>
    arrange(desc(mean)) |>
    head(1) |>
    pull(neighbors)

best_k

[1] 3

In [14]:
best_k_knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
    set_engine("kknn") |>
    set_mode("classification")

chr_best_k_wf <- workflow() |>
    add_recipe(chr_knn_recipe) |>
    add_model(best_k_knn_spec) |>
    fit(data = chr_training)

chr_best_k_result <- predict(chr_best_k_wf, chr_testing) |>
    bind_cols(chr_testing) |>
    metrics(truth = label, estimate = .pred_class)

chr_best_k_result

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.8138462
kap,multiclass,0.8063132


In [17]:
chr_best_k_result2 <- predict(chr_best_k_wf, chr_testing) |>
    bind_cols(chr_testing) |>
    conf_mat(truth = label, estimate = .pred_class)

chr_best_k_result2

          Truth
Prediction  a  b  c  d  e  f  g  h  i  j  k  l  m  n  o  p  q  r  s  t  u  v  w
         a 16  0  0  0  0  0  0  1  0  0  0  0  2  0  0  0  0  1  1  0  0  0  0
         b  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
         c  0  0 25  0  2  0  1  0  0  0  0  2  0  0  0  0  0  2  2  0  0  0  0
         d  0  1  0 19  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
         e  0  1  1  0 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
         f  0  0  0  0  0 21  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0
         g  0  2  1  0  0  0 21  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0
         h  0  0  0  0  0  0  1 18  0  0  1  0  1  3  0  0  0  0  0  0  1  0  1
         i  0  1  0  0  0  0  0  0 18  2  0  0  0  0  0  0  0  0  0  0  0  0  0
         j  0  0  0  0  0  0  1  0  0 24  0  0  0  0  0  0  1  0  1  1  1  0  0
         k  1  1  0  0  0  0  0  0  0  0 17  0  0  0  0  0  0  1  0  0  0  0  0
         l  0  0  0  0  